In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


In [2]:
"""
MNIST 데이터세트를 로드하고 준비합니다. 샘플 데이터를 정수에서 부동 소수점 숫자로 변환합니다.
"""
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
"""
층을 차례대로 쌓아 tf.keras.Sequential 모델을 만듭니다. 훈련에 사용할 옵티마이저(optimizer)와 손실 함수를 선택합니다:
"""
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
"""
각 예시에서 모델은 각 클래스에 대해 하나씩, logits 또는 log-odds 스코어 벡터를 반환합니다.
"""
predictions = model(x_train[:1]).numpy()
predictions

array([[0.07920178, 0.12940672, 0.09074259, 0.08143337, 0.08310716,
        0.12129363, 0.13972594, 0.12010849, 0.10762591, 0.04735447]],
      dtype=float32)

In [13]:
"""
tf.nn.softmax 함수는 다음과 같이 이러한 로짓을 각 클래스에 대한 확률로 변환합니다.
참고: tf.nn.softmax 함수를 네트워크의 마지막 레이어에 대한 활성화 함수로 베이킹할 수 있습니다. 이렇게 하면 모델 출력을 더 직접적으로 해석할 수 있지만 이 접근법은 소프트맥스 출력을 사용할 경우 모든 모델에 대해 정확하고 수치적으로 안정적인 손실 계산을 제공하는 것이 불가능하므로 권장하지 않습니다.
"""
tf.nn.softmax(predictions).numpy()

array([[0.09790612, 0.10294697, 0.09904259, 0.09812485, 0.09828923,
        0.10211514, 0.10401481, 0.10199419, 0.10072895, 0.0948372 ]],
      dtype=float32)

In [6]:
"""
losses.SparseCategoricalCrossentropy를 사용하여 로짓의 벡터와 True 인덱스를 사용하고 각 예시에 대해 스칼라 손실을 반환하는 훈련용 손실 함수를 정의합니다.
"""
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
"""
이 손실은 실제 클래스의 음의 로그 확률과 같습니다. 모델이 올바른 클래스를 확신하는 경우 손실은 0입니다.

이 훈련되지 않은 모델은 무작위에 가까운 확률(각 클래스에 대해 1/10)을 제공하므로 초기 손실은 -tf.math.log(1/10) ~= 2.3에 근접해야 합니다.
"""
loss_fn(y_train[:1], predictions).numpy()

2.2816544

In [8]:
"""
훈련을 시작하기 전에 Keras Model.compile을 사용하여 모델을 구성하고 컴파일합니다. optimizer 클래스를 adam으로 설정하고 loss를 앞에서 정의한 loss_fn 함수로 설정합니다. metrics 매개변수를 accuracy로 설정하여 모델에 대해 평가할 메트릭을 지정합니다.
"""
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
"""
모델을 훈련하고 평가합니다:
"""
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
   1/1875 [..............................] - ETA: 5:18 - loss: 2.2442 - accuracy: 0.1250

2024-11-30 21:01:58.437123: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/cyj/miniconda3/envs/Tensorflow/lib/python3.11/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 2s 759us/step - loss: 0.2989 - accuracy: 0.9136
Epoch 2/5
1875/1875 [==============================] - 1s 742us/step - loss: 0.1440 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 1s 795us/step - loss: 0.1071 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 1s 773us/step - loss: 0.0870 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 2s 801us/step - loss: 0.0729 - accuracy: 0.9768


In [10]:
"""
Model.evaluate 메서드는 일반적으로 "Validation-set" 또는 "Test-set"에서 모델 성능을 확인합니다.
훈련된 이미지 분류기는 이 데이터셋에서 약 98%의 정확도를 달성합니다. 더 자세한 내용은 TensorFlow 튜토리얼을 참고하세요.
"""
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 2s 867us/step - loss: 0.0673 - accuracy: 0.9787
Epoch 2/5
1875/1875 [==============================] - 2s 809us/step - loss: 0.0577 - accuracy: 0.9815
Epoch 3/5
1875/1875 [==============================] - 2s 908us/step - loss: 0.0526 - accuracy: 0.9830
Epoch 4/5
1875/1875 [==============================] - 1s 783us/step - loss: 0.0480 - accuracy: 0.9839
Epoch 5/5
1875/1875 [==============================] - 2s 828us/step - loss: 0.0443 - accuracy: 0.9854
313/313 - 0s - loss: 0.0699 - accuracy: 0.9797 - 195ms/epoch - 624us/step


[0.0699368268251419, 0.9797000288963318]

In [11]:
"""
모델이 확률을 반환하도록 하려면 다음과 같이 훈련된 모델을 래핑하고 여기에 소프트맥스를 첨부할 수 있습니다.
"""
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533732, 0.08533732, 0.08533733, 0.08534115, 0.08533732,
        0.08533732, 0.08533732, 0.23196018, 0.08533733, 0.08533739],
       [0.08533683, 0.08533698, 0.23196772, 0.08533737, 0.08533683,
        0.08533683, 0.08533683, 0.08533683, 0.08533683, 0.08533683],
       [0.08533856, 0.2319404 , 0.08533911, 0.08533859, 0.08533892,
        0.08533859, 0.08533873, 0.08534519, 0.08534331, 0.08533856],
       [0.23196739, 0.08533686, 0.08533695, 0.08533686, 0.08533686,
        0.08533689, 0.08533701, 0.08533693, 0.08533686, 0.0853374 ],
       [0.08546922, 0.08546907, 0.08546908, 0.08546907, 0.22986065,
        0.08546907, 0.0854691 , 0.08547482, 0.08546908, 0.08638087]],
      dtype=float32)>